In [ ]:
import os
import pandas as pd
import zipfile
import random
from tqdm import tqdm

In [ ]:
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

In [ ]:
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, LongType
import pyspark.sql.functions as F
from pyspark.sql.functions import isnan, when, count, col, avg, desc, asc


In [ ]:



# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","6g")\
        .getOrCreate()
spark.sparkContext

<SparkContext master=local[2] appName=Create-DataFrame>

In [ ]:
# !wget https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip


In [ ]:
!ls

data  recipe_reviews.zip  Untitled.ipynb


In [ ]:
# !mv recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip


In [ ]:
# with zipfile.ZipFile("recipe_reviews.zip", 'r') as zip_ref:
#     zip_ref.extractall("./data")

In [ ]:
!ls ./data

'Recipe Reviews and User Feedback Dataset.csv'


In [ ]:
!head -3 "data/Recipe Reviews and User Feedback Dataset.csv"

,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,1665619889,0,0,0,5,527,"I tweaked it a little, removed onions because of onion haters in my house, used Italian seasoning instead of just oregano, and use a paprika/ cayenne mix and a little more than the recipe called for.. we like everything a bit more hot. The chili was amazing! It was easy to make and everyone absolutely loved it. It will now be a staple meal in our house."
1,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,1665277687,0,7,0,5,724,"Bush used to have a white chili bean and it made this recipe super simple. I’ve written to them and asked them to please!, bring them back"


In [ ]:
dfs = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")


In [ ]:
dfs= dfs.withColumnRenamed('_c0', 'id')

In [ ]:
dfs.show(5)

+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
| id|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id| user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|   Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|   Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush used to have...|
|  2|          

25/01/21 11:03:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [ ]:
print(dfs.columns)

['id', 'recipe_number', 'recipe_code', 'recipe_name', 'comment_id', 'user_id', 'user_name', 'user_reputation', 'created_at', 'reply_count', 'thumbs_up', 'thumbs_down', 'stars', 'best_score', 'text']


In [ ]:
dfsc = df_reviews.na.drop()

In [ ]:
dfsc.orderBy('reply_count', ascending=False).show(10)

+---+-------------+-----------+--------------------+--------------------+--------------------+-----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|         recipe_name|          comment_id|             user_id|        user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+--------------------+--------------------+--------------------+-----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
| 17|          002|       3309|Best Ever Banana ...|sp_aUSaElGf_3309_...|u_1tOHujEFJQIEVu0...|     OrangeBowtie|              0|1622648873|          3|        5|          0|    5|       354|The title is not ...|
|  8|          003|       2832|   Cheeseburger Soup|sp_aUSaElGf_2832_...|      u_inTFTX8AEJ0X|   ladypenny36619|              1|1659170678|          3| 

25/01/21 11:13:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [ ]:
dfsc.groupBy('stars').count().show()


25/01/21 11:14:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


+-----+-----+
|stars|count|
+-----+-----+
|    3|  490|
|    0| 1696|
|    5|13829|
|    1|  280|
|    4| 1655|
|    2|  232|
+-----+-----+



In [ ]:
dfsc.groupBy('recipe_name').agg(F.count('comment_id').alias('comment_count')) \
  .orderBy('comment_count', ascending=False) \
  .show(10)

+--------------------+-------------+
|         recipe_name|comment_count|
+--------------------+-------------+
|   Cheeseburger Soup|          725|
|  Creamy White Chili|          654|
|Best Ever Banana ...|          509|
|Enchilada Casser-...|          421|
|Basic Homemade Bread|          397|
|Favorite Chicken ...|          395|
|Flavorful Chicken...|          368|
|Amish Breakfast C...|          338|
|Zucchini Pizza Ca...|          332|
|    Cauliflower Soup|          324|
+--------------------+-------------+
only showing top 10 rows



25/01/21 11:16:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv
